In [1]:
# import packages 

import pandas as pd
import numpy as np
import requests
import os
import pickle
import matplotlib.pyplot as plt

In [2]:
# adjust pandas printing

pd.options.display.max_colwidth = 200
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [3]:
# register for census API https://api.census.gov/data/key_signup.html
# store key as environment variable 

CENSUS_API_KEY = os.getenv('CENSUS_API_KEY')

In [4]:
# import dataframe redfin

dfrf = pd.read_pickle('Redfin_pickle')

In [5]:
# inspect data frame 

dfrf.head()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),Neighborhood,Location,dt,Crime Score
6,379900.0,2.0,1.5,1226.0,1742.0,1890.0,https://www.redfin.com/PA/Pittsburgh/159-1-2-38th-St-15201/home/74651930,Lawrenceville,"(-79.9655419, 40.4679636)",2021-09-01,165.417001
8,390000.0,3.0,3.0,1519.0,2178.0,1910.0,https://www.redfin.com/PA/Pittsburgh/4919-Hatfield-St-15201/home/73509681,Lawrenceville,"(-79.9573142, 40.4780842)",2020-12-01,148.431000
14,302500.0,5.0,2.0,3034.0,2178.0,1900.0,https://www.redfin.com/PA/Pittsburgh/259-45th-St-15201/home/74651854,Lawrenceville,"(-79.9576202, 40.4711406)",2021-06-01,237.810844
16,301000.0,3.0,1.5,1972.0,1306.0,1890.0,https://www.redfin.com/PA/Pittsburgh/431-Taylor-St-15224/home/74563342,Bloomfield,"(-79.9523315, 40.4605557)",2021-10-01,248.895336
24,396000.0,3.0,2.0,1462.0,4081.0,1926.0,https://www.redfin.com/PA/Pittsburgh/6529-Stanton-Ave-15206/home/74472408,Highland Park,"(-79.9115632, 40.4723218)",2022-08-01,62.360468


In [6]:
# get the census tract by latitude and longitude based on redfin data

import censusgeocode as cg 

def getCensusGeoData(latitude, longitude): 
    initResult = cg.coordinates(x=latitude, y=longitude).get('Census Tracts')[0]
    tract = initResult.get('TRACT') 
    county = initResult.get('COUNTY')    
    state = initResult.get('STATE') 
    return tract, county, state

In [7]:
# load pickle
dfrf = pd.read_pickle("./censusTract.pkl")  

# UNCOMMENT FOR LIVE MODEL RUNS 
# get target census tract from redfin data
# dfrf[['censusTract', 'censusCounty', 'censusState']] = dfrf.apply(lambda row: getCensusGeoData(row.Location[0], row.Location[1]), axis = 1, result_type="expand")

# set up key to join in census data
dfrf['joinKey'] = dfrf['censusState'] + dfrf['censusCounty'] + dfrf['censusTract']

# save to pickle
# dfrf.to_pickle("./censusTract.pkl")  

# inspect results
dfrf.head()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),Neighborhood,Location,dt,Crime Score,censusTract,censusCounty,censusState,joinKey
6,379900.0,2.0,1.5,1226.0,1742.0,1890.0,https://www.redfin.com/PA/Pittsburgh/159-1-2-38th-St-15201/home/74651930,Lawrenceville,"(-79.9655419, 40.4679636)",2021-09-01,165.417001,060300,003,42,42003060300
8,390000.0,3.0,3.0,1519.0,2178.0,1910.0,https://www.redfin.com/PA/Pittsburgh/4919-Hatfield-St-15201/home/73509681,Lawrenceville,"(-79.9573142, 40.4780842)",2020-12-01,148.431000,090100,003,42,42003090100
14,302500.0,5.0,2.0,3034.0,2178.0,1900.0,https://www.redfin.com/PA/Pittsburgh/259-45th-St-15201/home/74651854,Lawrenceville,"(-79.9576202, 40.4711406)",2021-06-01,237.810844,090200,003,42,42003090200
16,301000.0,3.0,1.5,1972.0,1306.0,1890.0,https://www.redfin.com/PA/Pittsburgh/431-Taylor-St-15224/home/74563342,Bloomfield,"(-79.9523315, 40.4605557)",2021-10-01,248.895336,080200,003,42,42003080200
24,396000.0,3.0,2.0,1462.0,4081.0,1926.0,https://www.redfin.com/PA/Pittsburgh/6529-Stanton-Ave-15206/home/74472408,Highland Park,"(-79.9115632, 40.4723218)",2022-08-01,62.360468,110600,003,42,42003110600


In [8]:
# https://www.census.gov/data/developers/data-sets/acs-1year.html
# https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
# reference query: https://api.census.gov/data/2019/acs/acs5?get=NAME,B23025_003E,B23025_005E,B15003_001E,B15003_002E,B15003_003E,B15003_004E,B15003_005E,B15003_006E,B15003_007E,B15003_008E,B15003_009E,B15003_010E,B15003_011E,B15003_012E,B15003_013E,B15003_014E,B15003_015E,B15003_016E&for=block+group:*&in=state:17+county:031
# reference query: webResponse = requests.get('https://api.census.gov/data/'+targetYear+'/acs/acs5?get=NAME,B08126_001E,B08126_002E,B08126_003E,B08126_004E,B08126_005E,B08126_006E,B08126_007E,B08126_008E,B08126_009E,B08126_010E,B08126_011E,B08126_012E,B08126_013E,B08126_014E,B08126_015E,B06009_001E,B06009_002E,B06009_003E,B06009_004E,B06009_005E,B06009_006E,B07010_001E,B07010_002E,B07010_004E,B07010_005E,B07010_006E,B07010_007E,B07010_008E,B07010_009E,B07010_010E,B07010_011E&for=tract:*&in=state:'+targetState+'+county:'+targetCounty+"&key="+CENSUS_API_KEY).json()
# representative map of census tract: https://www2.census.gov/geo/maps/DC2020/PL20/st42_pa/censustract_maps/c42003_allegheny/DC20CT_C42003.pdf

def getACSData(targetYear, targetCounty, targetState):
    webResponse = requests.get('https://api.census.gov/data/'+targetYear+'/acs/acs5?get=NAME,B01002_001E,B15003_001E,B15003_002E,B15003_016E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,B19083_001E&for=tract:*&in=state:'+targetState+'+county:'+targetCounty+"&key="+CENSUS_API_KEY).json()
    df = pd.DataFrame.from_records(webResponse)
    df.columns = df.iloc[0] # enforce columns
    df = df[1:] # keep data but have properly formatted columns from index 0 
    df['joinKey'] = df['state'] + df['county'] + df['tract'] # set up join key to merge with geoData and redfin
    df.drop(columns = ['state', 'county', 'tract', 'NAME'], inplace=True)
    df.rename(columns={'B01002_001E':'age_Median',
                       'B15003_001E':'education_Total',
                       'B15003_002E':'education_NoSchoolingCompleted',
                       'B15003_016E':'education_12thgradeNoDiploma',
                       'B15003_017E':'education_RegularHighSchoolDiploma',
                       'B15003_018E':'education_GEDorAlternativeCredential',
                       'B15003_019E':'education_SomeCollegeLessThan1year',
                       'B15003_020E':'education_SomeCollege',
                       'B15003_021E':'education_AssociateDegree',
                       'B15003_022E':'education_BachelorDegree',
                       'B15003_023E':'education_MasterDegree',
                       'B15003_024E':'education_ProfessionalSchoolDegree',
                       'B15003_025E':'education_DoctorateDegree',
                       'B19001_001E':'income_Total',
                       'B19001_002E':'income_LessThan10k',
                       'B19001_003E':'income_10Kto15K',
                       'B19001_004E':'income_15Kto20K',
                       'B19001_005E':'income_20Kto25K',
                       'B19001_006E':'income_25Kto30K',
                       'B19001_007E':'income_30Kto35K',
                       'B19001_008E':'income_35Kto40K',
                       'B19001_009E':'income_40Kto45K',
                       'B19001_010E':'income_45Kto50K',
                       'B19001_011E':'income_55Kto60K',
                       'B19001_012E':'income_60Kto75K',
                       'B19001_013E':'income_75Kto100K',
                       'B19001_014E':'income_100Kto125K',
                       'B19001_015E':'income_125Kto150K',
                       'B19001_016E':'income_150Kto200K',
                       'B19001_017E':'income_200KOrMore',
                       'B19083_001E':'inequality_GiniIndex'}, inplace = True)
    # convert data types
    df.education_Total = df.education_Total.astype(int)
    df.education_NoSchoolingCompleted = df.education_NoSchoolingCompleted.astype(int)
    df.education_12thgradeNoDiploma = df.education_12thgradeNoDiploma.astype(int)
    df.education_RegularHighSchoolDiploma = df.education_RegularHighSchoolDiploma.astype(int)
    df.education_GEDorAlternativeCredential = df.education_GEDorAlternativeCredential.astype(int)
    df.education_SomeCollegeLessThan1year = df.education_SomeCollegeLessThan1year.astype(int)
    df.education_SomeCollege = df.education_SomeCollege.astype(int)
    df.education_AssociateDegree = df.education_AssociateDegree.astype(int)
    df.education_BachelorDegree = df.education_BachelorDegree.astype(int)
    df.education_MasterDegree = df.education_MasterDegree.astype(int)
    df.education_ProfessionalSchoolDegree = df.education_ProfessionalSchoolDegree.astype(int)
    df.education_DoctorateDegree = df.education_DoctorateDegree.astype(int)
    df.income_Total = df.income_Total.astype(int)
    df.income_LessThan10k = df.income_LessThan10k.astype(int)
    df.income_10Kto15K = df.income_10Kto15K.astype(int)
    df.income_15Kto20K = df.income_15Kto20K.astype(int)
    df.income_20Kto25K = df.income_20Kto25K.astype(int)
    df.income_25Kto30K = df.income_25Kto30K.astype(int)
    df.income_30Kto35K = df.income_30Kto35K.astype(int)
    df.income_35Kto40K = df.income_35Kto40K.astype(int)
    df.income_40Kto45K = df.income_40Kto45K.astype(int)
    df.income_45Kto50K = df.income_45Kto50K.astype(int)
    df.income_55Kto60K = df.income_55Kto60K.astype(int)
    df.income_60Kto75K = df.income_60Kto75K.astype(int)
    df.income_75Kto100K = df.income_75Kto100K.astype(int)
    df.income_100Kto125K = df.income_100Kto125K.astype(int)
    df.income_125Kto150K = df.income_125Kto150K.astype(int)
    df.income_150Kto200K = df.income_150Kto200K.astype(int)
    df.income_200KOrMore = df.income_200KOrMore.astype(int)
    # calculate percentages instead of raw numbers
    df.education_NoSchoolingCompleted /= df.education_Total
    df.education_12thgradeNoDiploma /= df.education_Total
    df.education_RegularHighSchoolDiploma /= df.education_Total
    df.education_GEDorAlternativeCredential /= df.education_Total
    df.education_SomeCollegeLessThan1year /= df.education_Total
    df.education_SomeCollege /= df.education_Total
    df.education_AssociateDegree /= df.education_Total
    df.education_BachelorDegree /= df.education_Total 
    df.education_MasterDegree /= df.education_Total
    df.education_ProfessionalSchoolDegree /= df.education_Total
    df.education_DoctorateDegree /= df.education_Total
    df.income_LessThan10k /= df.income_Total
    df.income_10Kto15K /= df.income_Total
    df.income_15Kto20K /= df.income_Total
    df.income_20Kto25K /= df.income_Total
    df.income_25Kto30K /= df.income_Total
    df.income_30Kto35K /= df.income_Total
    df.income_35Kto40K /= df.income_Total
    df.income_40Kto45K /= df.income_Total
    df.income_45Kto50K /= df.income_Total
    df.income_55Kto60K /= df.income_Total
    df.income_60Kto75K /= df.income_Total
    df.income_75Kto100K /= df.income_Total
    df.income_100Kto125K /= df.income_Total
    df.income_125Kto150K /= df.income_Total
    df.income_150Kto200K /= df.income_Total
    df.income_200KOrMore /= df.income_Total
    # drop redundant columns
    df = df.drop(columns=['income_Total'])
    return df

In [9]:
# check for numerous counties ... as the data scales so will this list
distinctStateCounties = list(set(dfrf['censusState'] + dfrf['censusCounty']))

distinctStateCounties

['42003']

In [10]:
# set up list that will hold dataframe objects
censusDFList = []

# iterate and populate list with dataframe objects
for i in range(len(distinctStateCounties)):
    print('Evaluating: ','2020', distinctStateCounties[i][2:], distinctStateCounties[i][:2])
    censusDFList.append(getACSData('2020', distinctStateCounties[i][2:], distinctStateCounties[i][:2]))
    
# turn list of dataframe objects into single dataframe since indexes will be shared
dfcensus = pd.concat(censusDFList)

Evaluating:  2020 003 42


In [11]:
# merge the census and redfin data together 

combined = dfrf.merge(dfcensus, on='joinKey', how='left')

In [12]:
# inspect data frame

combined.head()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),Neighborhood,Location,dt,Crime Score,censusTract,censusCounty,censusState,joinKey,age_Median,education_Total,education_NoSchoolingCompleted,education_12thgradeNoDiploma,education_RegularHighSchoolDiploma,education_GEDorAlternativeCredential,education_SomeCollegeLessThan1year,education_SomeCollege,education_AssociateDegree,education_BachelorDegree,education_MasterDegree,education_ProfessionalSchoolDegree,education_DoctorateDegree,income_LessThan10k,income_10Kto15K,income_15Kto20K,income_20Kto25K,income_25Kto30K,income_30Kto35K,income_35Kto40K,income_40Kto45K,income_45Kto50K,income_55Kto60K,income_60Kto75K,income_75Kto100K,income_100Kto125K,income_125Kto150K,income_150Kto200K,income_200KOrMore,inequality_GiniIndex
0,379900.0,2.0,1.5,1226.0,1742.0,1890.0,https://www.redfin.com/PA/Pittsburgh/159-1-2-38th-St-15201/home/74651930,Lawrenceville,"(-79.9655419, 40.4679636)",2021-09-01,165.417001,060300,003,42,42003060300,33.4,2019,0.006439,0.000000,0.114908,0.045072,0.016345,0.082714,0.011392,0.383358,0.187221,0.075285,0.025755,0.049392,0.013678,0.051672,0.018997,0.028116,0.065350,0.000000,0.060790,0.018997,0.055471,0.093465,0.177052,0.132219,0.044073,0.069149,0.121581,0.4399
1,390000.0,3.0,3.0,1519.0,2178.0,1910.0,https://www.redfin.com/PA/Pittsburgh/4919-Hatfield-St-15201/home/73509681,Lawrenceville,"(-79.9573142, 40.4780842)",2020-12-01,148.431000,090100,003,42,42003090100,33.2,1856,0.011853,0.015086,0.131466,0.005388,0.030172,0.061961,0.042026,0.285022,0.164332,0.101832,0.029634,0.012542,0.030936,0.020903,0.040970,0.049331,0.053512,0.057692,0.021739,0.000000,0.088629,0.096990,0.114548,0.104515,0.031773,0.137960,0.137960,0.4843
2,302500.0,5.0,2.0,3034.0,2178.0,1900.0,https://www.redfin.com/PA/Pittsburgh/259-45th-St-15201/home/74651854,Lawrenceville,"(-79.9576202, 40.4711406)",2021-06-01,237.810844,090200,003,42,42003090200,37.1,2429,0.000000,0.004117,0.163442,0.012351,0.007822,0.076575,0.049403,0.340881,0.142857,0.088514,0.040758,0.057884,0.091816,0.061211,0.025283,0.021291,0.069860,0.007984,0.033932,0.054558,0.063207,0.067864,0.161677,0.073187,0.055223,0.099135,0.055888,0.4614
3,301000.0,3.0,1.5,1972.0,1306.0,1890.0,https://www.redfin.com/PA/Pittsburgh/431-Taylor-St-15224/home/74563342,Bloomfield,"(-79.9523315, 40.4605557)",2021-10-01,248.895336,080200,003,42,42003080200,32.9,1499,0.037358,0.000000,0.186791,0.054703,0.046031,0.073382,0.108072,0.248165,0.136091,0.010007,0.051368,0.073501,0.020309,0.018375,0.048356,0.011605,0.090909,0.135397,0.068665,0.059961,0.089942,0.062863,0.117988,0.077369,0.030948,0.069632,0.024178,0.4501
4,396000.0,3.0,2.0,1462.0,4081.0,1926.0,https://www.redfin.com/PA/Pittsburgh/6529-Stanton-Ave-15206/home/74472408,Highland Park,"(-79.9115632, 40.4723218)",2022-08-01,62.360468,110600,003,42,42003110600,42.7,1816,0.014868,0.000000,0.133811,0.000000,0.020374,0.049009,0.060573,0.271476,0.232930,0.128304,0.075991,0.030115,0.077945,0.030115,0.007086,0.027458,0.016829,0.026572,0.048716,0.007972,0.065545,0.135518,0.116032,0.082374,0.063773,0.095660,0.168291,0.4699


In [13]:
combined.columns

Index(['PRICE', 'BEDS', 'BATHS', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'Neighborhood', 'Location', 'dt', 'Crime Score', 'censusTract',
       'censusCounty', 'censusState', 'joinKey', 'age_Median',
       'education_Total', 'education_NoSchoolingCompleted',
       'education_12thgradeNoDiploma', 'education_RegularHighSchoolDiploma',
       'education_GEDorAlternativeCredential',
       'education_SomeCollegeLessThan1year', 'education_SomeCollege',
       'education_AssociateDegree', 'education_BachelorDegree',
       'education_MasterDegree', 'education_ProfessionalSchoolDegree',
       'education_DoctorateDegree', 'income_LessThan10k', 'income_10Kto15K',
       'income_15Kto20K', 'income_20Kto25K', 'income_25Kto30K',
       'income_30Kto35K', 'income_35Kto40K', 'income_40Kto45K',
       'income_45Kto50K', 'income_55Kto60K', 'income_60Kto75K',
       'income_75Kto100K', 'inco

In [14]:
# save to pickle
combined.to_pickle("./censusMergedWithRedfin.pkl")  

Overview of census data tables and interesting features<br><br>
https://data.census.gov/all/tables?d=ACS+5-Year+Estimates+Detailed+Tables -> search for terms
https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.2020.html#list-tab-9CLUDMSRO7QCV1OP40 -> table shells<br>
https://www.census.gov/programs-surveys/acs/data/data-tables/table-ids-explained.html -> explain what table schema means<br><br>
B01001 SEX BY AGE<br>
B01002 MEDIAN AGE BY SEX - *used in model*<br>
B01003 TOTAL POPULATION - *used in model*<br>
B02001 RACE<br>
B11001 HOUSEHOLD TYPE (INCLUDING LIVING ALONE)<br>
B14001 SCHOOL ENROLLMENT BY LEVEL OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER - *used in model*<br>
B15003 EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER - *used in model*<br>
B19001 HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2020 INFLATION-ADJUSTED DOLLARS) - *used in model*<br>
B19013 MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2020 INFLATION-ADJUSTED DOLLARS)<br>
B19083 GINI INDEX OF INCOME INEQUALITY - *used in model*<br>

B01002_001,B01003_001,B14001_001,B14001_002,B14001_003,B14001_004,B14001_005,B14001_006,B14001_007,B14001_008,B14001_009,B14001_010,B15003_001,B15003_002,B15003_003,B15003_004,B15003_005,B15003_006,B15003_007,B15003_008,B15003_009,B15003_010,B15003_011,B15003_012,B15003_013,B15003_014,B15003_015,B15003_016,B15003_017,B15003_018,B15003_019,B15003_020,B15003_021,B15003_022,B15003_023,B15003_024,B15003_025,B19001_001,B19001_002,B19001_003,B19001_004,B19001_005,B19001_006,B19001_007,B19001_008,B19001_009,B19001_010,B19001_011,B19001_012,B19001_013,B19001_014,B19001_015,B19001_016,B19001_017,B19083_001

'B01002_001':'age_Median',
'B01003_001':'population_Total',
'B14001_001':'education_Total',
'B14001_002':'education_EnrolledInSchool:',
'B14001_003':'education_EnrolledInNurserySchool',
'B14001_004':'education_EnrolledInKindergarten',
'B14001_005':'education_EnrolledInGrade1ToGrade4',
'B14001_006':'education_EnrolledInGrade5ToGrade8',
'B14001_007':'education_EnrolledInGrade9ToGrade12',
'B14001_008':'education_EnrolledInCollege',
'B14001_009':'education_GraduateOrProfessionalSchool',
'B14001_010':'education_NotEnrolledInSchool',
'B15003_001':'popEducation_Total',
'B15003_002':'popEducation_NoSchoolingCompleted',
'B15003_003':'popEducation_NurserySchool',
'B15003_004':'popEducation_Kindergarten',
'B15003_005':'popEducation_1stgrade',
'B15003_006':'popEducation_2ndgrade',
'B15003_007':'popEducation_3rdgrade',
'B15003_008':'popEducation_4thgrade',
'B15003_009':'popEducation_5thgrade',
'B15003_010':'popEducation_6thgrade',
'B15003_011':'popEducation_7thgrade',
'B15003_012':'popEducation_8thgrade',
'B15003_013':'popEducation_9thgrade',
'B15003_014':'popEducation_10thgrade',
'B15003_015':'popEducation_11thgrade',
'B15003_016':'popEducation_12thgradeNoDiploma',
'B15003_017':'popEducation_RegularHighSchoolDiploma',
'B15003_018':'popEducation_GEDorAlternativeCredential',
'B15003_019':'popEducation_SomeCollegeLessThan1year',
'B15003_020':'popEducation_SomeCollege,
'B15003_021':'popEducation_AssociateDegree,
'B15003_022':'popEducation_BachelorDegree',
'B15003_023':'popEducation_MasterDegree',
'B15003_024':'popEducation_ProfessionalSchoolDegree',
'B15003_025':'popEducation_DoctorateDegree,
'B19001_001':'income_Total',
'B19001_002':'income_LessThan10k',
'B19001_003':'income_10Kto15K',
'B19001_004':'income_15Kto20K',
'B19001_005':'income_20Kto25K',
'B19001_006':'income_25Kto30K,
'B19001_007':'income_30Kto35K
'B19001_008':'income_35Kto40K',
'B19001_009':'income_40Kto45K',
'B19001_010':'income_45Kto50K',
'B19001_011':'income_55Kto60K',
'B19001_012':'income_60Kto75K,
'B19001_013':'income_75Kto100K,
'B19001_014':'income_100Kto125K,
'B19001_015':'income_125Kto150K,
'B19001_016':'income_150Kto200K,
'B19001_017':'income_200KOrMore,
'B19083_001':'inequality_GiniIndex'